In [ ]:
%config InlineBackend.figure_format = 'svg'

In [1]:
import networkx as nx
#from node2vec import Node2Vec
import os
import pickle
import random
from itertools import chain
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from random import sample
import seaborn as sns
import numpy as np

## Functions

In [2]:
def load_pickle(project_folder, prefix):
    fname = os.path.join(project_folder, '%s.pkl' % prefix)
    with open(fname, 'rb') as fh:
        return pickle.load(fh)

In [3]:
working_dir = "data/vectors/"

mg=load_pickle(project_folder=working_dir ,prefix='multi_graph')

## Read and merge covid19 literature genes

In [4]:
cov19_lit_genes_lee = pd.read_csv(working_dir + 'covid19-lit/Lee.txt', sep=" ")
cov19_lit_genes_gordon = pd.read_csv(working_dir + 'covid19-lit/Gordon_sarscov2.txt', sep=" ")
cov19_lit_genes_overmayer = pd.read_csv(working_dir + 'covid19-lit/Overmyer.csv')
cov19_lit_genes_pascal = pd.read_csv(working_dir + 'covid19-lit/Pascal_unique_interactors.csv', sep=";")#which column?
cov19_lit_genes_schneider = pd.read_csv(working_dir + 'covid19-lit/Schneider.csv')
cov19_lit_genes_singh = pd.read_csv(working_dir + 'covid19-lit/Singh.csv')
cov19_lit_genes_wei = pd.read_csv(working_dir + 'covid19-lit/Wei.csv')
lit_genes =  []
lit_genes.append(cov19_lit_genes_lee["Protein"].tolist())
lit_genes.append(cov19_lit_genes_gordon["Gene"].tolist())
lit_genes.append(cov19_lit_genes_pascal["unique_human_interactors"].tolist())
lit_genes.append(cov19_lit_genes_schneider["Gene"].tolist())
lit_genes.append(cov19_lit_genes_wei["Gene"].tolist())
lit_genes.append(cov19_lit_genes_singh["receptors"].tolist())

In [5]:
cov19_lit_genes_overmayer

,biomolecule_id,standardized_name,biomolecule_type,Log2(COVID-19/non-COVID-19),q_value for COVID-19 association,q_value for HFD-45 association,Elastic_Net_coefficient predicting HFD-45
0,7662,CRTAC1,protein,-2.444875,1.410000e-06,4.920000e-06,0.333678
1,8336,APOA2,protein,-0.835548,5.690000e-06,1.260000e-08,0.266402
2,7921,AHSG,protein,-0.718041,3.600000e-06,5.970000e-07,0.156317
3,7974,GSN,protein,-0.655304,1.170000e-07,5.030000e-08,0.501908
4,8298,PGLYRP2,protein,-0.650154,1.415400e-04,6.390000e-07,0.006761
...,...,...,...,...,...,...,...
105,11167,CEACAM8,transcript,2.232000,1.000000e-09,6.260000e-06,-0.041290
106,12536,DEFA4,transcript,2.245000,1.000000e-09,7.590000e-05,-0.044751
107,19547,OLFM4,transcript,2.369000,3.460000e-05,3.480000e-05,-0.057111
108,18189,MMP8,transcript,2.534000,1.000000e-09,9.340000e-07,-0.156506


In [6]:
unique_genes = {x for l in lit_genes for x in l}
len(unique_genes) #860

860

In [7]:
unique_genes= sorted(unique_genes)

with open(working_dir + 'unique_covid_genes_combined.txt', 'w') as f:
    for val in unique_genes:
        f.write(val)
        f.write("\n")

In [8]:
covid_genes = pd.read_csv(working_dir + "unique_covid_genes_combined.txt", names=['covidGenes'])

In [9]:
final_non_gene_connections = pd.read_csv("data/covid_net/meanSim_all/meanSim_robust_all.txt")

In [11]:
final_non_gene_connections[final_non_gene_connections["node1"]=="SMARCA5"]

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim
837536,SMARCA5,RBBP4,1,gene,gene,100,0.875941,0.906647,0.014036
837537,SMARCA5,DNMT3A,2,gene,gene,100,0.855644,0.891656,0.023140
837538,SMARCA5,SPI1,2,gene,gene,100,0.847635,0.878376,0.015455
837539,SMARCA5,MTA3,2,gene,gene,100,0.830777,0.867243,0.014053
837540,SMARCA5,GATAD2A,2,gene,gene,100,0.830023,0.877552,0.018835
...,...,...,...,...,...,...,...,...,...
837779,SMARCA5,RPL7,2,gene,gene,100,0.650588,0.725214,0.034514
837780,SMARCA5,RAD23A,2,gene,gene,100,0.650511,0.697849,0.022297
837781,SMARCA5,SAP30,2,gene,gene,100,0.650330,0.723942,0.031557
837782,SMARCA5,NFATC1,2,gene,gene,100,0.650171,0.700081,0.022618


In [13]:
final_non_gene_connections.sort_values(by=['max_sim'])

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim
984429,USP53,FYTTD1,2,gene,gene,89,0.651186,0.670941,0.009841
772728,RPL30,DCAF4L1,2,gene,gene,100,0.653440,0.673070,0.010208
647480,PGR,YAE1D1,2,gene,gene,100,0.650443,0.673582,0.013729
700782,PSMA3,IQCE,1,gene,gene,100,0.652831,0.673590,0.011116
435683,IQCE,PSMA3,1,gene,gene,100,0.652831,0.673590,0.011116
...,...,...,...,...,...,...,...,...,...
557034,MYH7B,MYOT,2,gene,gene,100,0.997536,0.999537,0.000998
560615,MYOT,NEB,2,gene,gene,100,0.997581,0.999597,0.000999
582190,NEB,MYOT,2,gene,gene,100,0.997581,0.999597,0.000999
810722,SERPINA5,MYOT,2,gene,gene,100,0.997622,0.999625,0.000949


## Add Annotation to the data frame

In [42]:
# Add literature source as a new column
source = []
assigned = "F"
sources_list = ""
for gene in final_non_gene_connections["node1"].tolist():
   # gene = gene[3:]
    if gene in cov19_lit_genes_lee["Protein"].tolist():
        sources_list ="Lee"
        assigned = "T"
    if gene in cov19_lit_genes_gordon["Gene"].tolist():
        if assigned == "F": 
            sources_list = "Gordon"
            assigned = "T"
        else:
            sources_list+="+Gordon"
    if gene in cov19_lit_genes_pascal["unique_human_interactors"].tolist():
        if assigned == "F": 
            sources_list = "Pascal"
            assigned = "T"
        else:
            sources_list+="+Pascal"
    assigned = "F"
    source.append((gene, sources_list))
    sources_list= ""

#non_genes_neigh["source"] = 

In [43]:
final_non_gene_connections["source"] = [item[1] for item in source]
final_non_gene_connections['source'].replace('', np.nan, inplace=True)
final_non_gene_connections.dropna(subset=['source'], inplace=True)

In [44]:
final_non_gene_connections['experiment']="physical_binding"

In [45]:
final_non_gene_connections.sort_values(by=['node1'])

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim,source,experiment
419,AAR2,ABCE1,2,gene,gene,100,0.863328,0.896865,0.021704,Gordon,physical_binding
521,AAR2,USP39,3,gene,gene,100,0.660058,0.746561,0.026448,Gordon,physical_binding
520,AAR2,NEK7,2,gene,gene,100,0.660512,0.747091,0.037465,Gordon,physical_binding
519,AAR2,PIH1D1,2,gene,gene,100,0.660770,0.740028,0.036551,Gordon,physical_binding
518,AAR2,EXOSC6,2,gene,gene,100,0.662369,0.724169,0.023866,Gordon,physical_binding
...,...,...,...,...,...,...,...,...,...,...,...
1048649,ZYG11B,HSPBAP1,2,gene,gene,100,0.712752,0.814477,0.043851,Gordon,physical_binding
1048648,ZYG11B,TRIM69,2,gene,gene,100,0.712785,0.788200,0.033193,Gordon,physical_binding
1048647,ZYG11B,CUL9,2,gene,gene,100,0.713541,0.768151,0.026555,Gordon,physical_binding
1048661,ZYG11B,FIG4,2,gene,gene,100,0.708239,0.763244,0.020804,Gordon,physical_binding


In [46]:
final_non_gene_connections.to_csv("results/covid_net/edgelist/phy_bind_meanSim_all.csv")

# CRISPR

In [44]:
final_non_gene_connections = pd.read_csv("results/covid_net/meanSim_all/meanSim_robust_nongenes.txt")

In [32]:
final_non_gene_connections

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim
0,CCHCR1,CCDC18,2.0,gene,gene,100,0.875779,0.919562,0.015228
1,CCHCR1,DTNBP1,1.0,gene,gene,100,0.831413,0.877013,0.018870
2,CCHCR1,NEFH,2.0,gene,gene,100,0.829967,0.866806,0.023422
3,CCHCR1,AZI2,2.0,gene,gene,100,0.824747,0.879500,0.032735
4,CCHCR1,EXOC3,2.0,gene,gene,100,0.819404,0.844295,0.013389
...,...,...,...,...,...,...,...,...,...
6057633,ZSCAN9,USO1,2.0,gene,gene,1,0.643092,0.643092,NaN
6057634,ZSCAN9,HK3,2.0,gene,gene,1,0.642268,0.642268,NaN
6057635,ZSCAN9,COG6,2.0,gene,gene,1,0.641416,0.641416,NaN
6057636,ZSCAN9,TGFB3,2.0,gene,gene,1,0.641027,0.641027,NaN


In [48]:
# Add literature source as a new column
source = []
assigned = "F"
sources_list = ""
for gene in final_non_gene_connections["node1"].tolist():
    #gene = gene[3:]

    if gene in cov19_lit_genes_schneider["Gene"].tolist(): 
        sources_list = "Schneider"
        assigned = "T"
    if gene in cov19_lit_genes_wei["Gene"].tolist():
        if assigned == "F": 
            sources_list = "Wei"
            assigned = "T"
        else:
            sources_list+="+Wei"

    assigned = "F"
    source.append((gene, sources_list))
    sources_list= ""
    

#non_genes_neigh["source"] = 

In [49]:
final_non_gene_connections["source"] = [item[1] for item in source]
final_non_gene_connections['source'].replace('', np.nan, inplace=True)
final_non_gene_connections.dropna(subset=['source'], inplace=True)

In [50]:
final_non_gene_connections

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim,source
39022,ANKRD13D,CYS1,2,gene,gene,100,0.850520,0.883507,0.026591,Schneider
39023,ANKRD13D,GP1BA,2,gene,gene,100,0.826213,0.880150,0.029057,Schneider
39024,ANKRD13D,OTUD3,2,gene,gene,100,0.824804,0.857011,0.014004,Schneider
39025,ANKRD13D,RNF216,2,gene,gene,100,0.819279,0.877285,0.029509,Schneider
39026,ANKRD13D,VAMP3,2,gene,gene,100,0.808047,0.861466,0.024768,Schneider
...,...,...,...,...,...,...,...,...,...,...
1045968,ZNHIT2,NUP188,2,gene,gene,99,0.660068,0.723076,0.037082,Schneider
1045969,ZNHIT2,ITGB1BP1,2,gene,gene,99,0.652251,0.760483,0.062503,Schneider
1045970,ZNHIT2,FXN,2,gene,gene,97,0.663401,0.740366,0.046435,Schneider
1045971,ZNHIT2,NUP153,2,gene,gene,96,0.655765,0.762518,0.064050,Schneider


In [51]:
final_non_gene_connections['experiment']="CRISPR"

In [52]:
final_non_gene_connections.to_csv("results/covid_net/edgelist/CRISPR_meanSim_all.csv")